In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from botorch.models.transforms.input import AffineInputTransform

from Neural_Net_Classes import NN as NN

import numpy as np
#import transformer

from Neural_Net_Classes import NN as NN
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch.nn.functional as F


In [ ]:
# Load the simulaiton CSV file
sim_training_set_df = pd.read_csv('../../simulation_data/simulation_data.csv')

# Load the experimental train and test sets
expt_training_set_df = pd.read_csv('../../experimental_data/training_set_1.csv')
expt_test_set_df = pd.read_csv('../../experimental_data/test_set_1.csv')

In [ ]:
# Access the arrays
sim_z_training_set = sim_training_set_df['z_target_um'].values
sim_TOD_training_set = sim_training_set_df['TOD_fs3'].values
sim_protons_training_set = sim_training_set_df['n_protons'].values

plt.clf()
ax = plt.figure().add_subplot(projection='3d')

ax.scatter( sim_TOD_training_set, sim_z_training_set, sim_protons_training_set, c='r',alpha=0.3, label='Simulation training Set')
ax.view_init(elev=40., azim=40)
plt.xlabel('TOD')
plt.ylabel('z_target')
plt.legend()

<h2> Normalize with Affine Input Transformer

In [ ]:

# Define the input and output normalizations, based on the training set from experiments
X = torch.tensor(expt_training_set_df[['z_target_um', 'TOD_fs3','GVD']].values, dtype=torch.float)
input_transform = AffineInputTransform(3, coefficient=X.std(axis=0), offset=X.mean(axis=0))
y = torch.tensor(expt_training_set_df['n_protons'].values, dtype=torch.float).reshape(-1,1)
output_transform = AffineInputTransform( 1, coefficient=y.std(axis=0), offset=y.mean(axis=0))
if (min(X.mean(axis=0)) == 0):
    print("Mean value used for normalization is 0. This will lead to NaNs ",X.mean(axis=0))
if (min(X.std(axis=0)) == 0):
    print("RMS value used for normalization is 0. This will lead to NaNs ", X.std(axis=0))


In [ ]:
# Apply normalization to the training and test sets
norm_sim_training_set_df = sim_training_set_df.copy()
norm_sim_training_set_df[['z_target_um', 'TOD_fs3','GVD']] = input_transform( torch.tensor( sim_training_set_df[['z_target_um', 'TOD_fs3','GVD']].values ) )
norm_sim_training_set_df[['n_protons']] = output_transform( torch.tensor( sim_training_set_df[['n_protons']].values.reshape(-1,1) ) )




In [ ]:
# Create a 3D plot
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.scatter( norm_sim_training_set_df['TOD_fs3'], norm_sim_training_set_df['z_target_um'],
           norm_sim_training_set_df['n_protons'], c='r', alpha=0.7, label='Training Set')

ax.view_init(elev=40., azim=40, roll=0)
# Set labels and title
ax.set_xlabel('Normalized TOD')
ax.set_ylabel('Normalized Z')
ax.set_zlabel('Normalized Protons')

# Add legend
ax.legend()
# Show plot
plt.show()

In [ ]:
# Create a 2D plot
fig = plt.figure()
ax = fig.add_subplot()

ax.scatter(norm_sim_training_set_df['z_target_um'],
           norm_sim_training_set_df['n_protons'], c='r', alpha=0.7, label='Training Set')

# Set labels and title
ax.set_xlabel('Normalized z')
ax.set_ylabel('Normalized protons')

# Add legend
ax.legend()
# Show plot
plt.show()

In [ ]:
norm_sim_inputs_training = torch.tensor( norm_sim_training_set_df[['z_target_um', 'TOD_fs3','GVD']].values, dtype=torch.float)
norm_sim_outputs_training = torch.tensor( norm_sim_training_set_df['n_protons'].values.reshape(-1,1), dtype=torch.float)



<h2> Train base neural net

In [ ]:
sim_net = NN()
sim_net.train_model(norm_sim_inputs_training, norm_sim_outputs_training,num_epochs=10000)
sim_net.plot_loss()

In [ ]:
sim_train_predictions = sim_net.predict(norm_sim_inputs_training)

<h2> Visualize NN prediction on same data that it was training on

In [ ]:
fig, ax = plt.subplots()

ax.scatter(norm_sim_training_set_df['z_target_um'], norm_sim_training_set_df['n_protons'], label='Simulation training set')

ax.scatter(norm_sim_training_set_df['z_target_um'], sim_train_predictions, label='predictions', s=50, facecolors='none', edgecolors='r')

plt.title("n_protons predictions")
plt.xlabel('z_target (m)')
plt.ylabel('Number of protons (1/sr)')
plt.legend()

In [ ]:
# Create a 3D plot
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Scatter plot for training set
ax.scatter(norm_sim_training_set_df['z_target_um'], norm_sim_training_set_df['TOD_fs3'], norm_sim_training_set_df['n_protons'], label='Training Set 1', alpha=0.7)


ax.scatter(norm_sim_training_set_df['z_target_um'], norm_sim_training_set_df['TOD_fs3'], sim_train_predictions, label='predictions 1', s=50, facecolors='none', edgecolors='r')
#ax.scatter(test_predictions['TOD'], test_predictions['Z_target'], test_predictions['predictions'], s=50, facecolors='none', edgecolors='r')

ax.view_init(elev=40., azim=40)
# Set labels and title
ax.set_title('Simulation Data v Predictions')
ax.set_xlabel('TOD')
ax.set_ylabel('z_target')
ax.set_zlabel('n Protons')

# Add legend
ax.legend()
# Show plot
plt.show()

<h2> Define NN that adds linear calibration parameters to output obtained from pre-trained base neural net </h2>

In [ ]:

class finetune_NN(NN):
    def __init__(self, base_model, hidden_size=20, 
                 learning_rate=0.001, patience=100, factor=0.5, threshold=1e-4):
        super(finetune_NN, self).__init__(hidden_size, learning_rate, patience, factor, threshold)
        
        # Copying weights from base_model to this model
        self.hidden1.load_state_dict(base_model.hidden1.state_dict())
        self.hidden2.load_state_dict(base_model.hidden2.state_dict())
        self.hidden3.load_state_dict(base_model.hidden3.state_dict())
        self.hidden4.load_state_dict(base_model.hidden4.state_dict())
        self.hidden5.load_state_dict(base_model.hidden5.state_dict())        
        self.output.load_state_dict(base_model.output.state_dict())
        # This is the weight and bias to be tuned on the output obtained from base model
        self.w2 =torch.nn.Parameter(torch.rand(1))
        self.b2 = torch.nn.Parameter(torch.rand(1))
        self.relu = nn.ReLU()
        self.criterion = nn.MSELoss()
        self.optimizer = optim.Adam(self.parameters(), lr=learning_rate)
        self.scheduler = ReduceLROnPlateau(self.optimizer, 'min', 
                                           factor=factor, patience=patience, threshold=threshold)
        
    def forward(self, x):
        x = self.relu(self.hidden1(x))
        x = self.relu(self.hidden2(x))
        x = self.relu(self.hidden3(x))
        x = self.relu(self.hidden4(x))
        x = self.relu(self.hidden5(x))
        x = self.output(x)
        x = x * self.w2 + self.b2
        
        return x


    def train_model(self, inputs, outputs, num_epochs=1500):
        '''
        args:
            tensor inputs input dataset to train NN
            tensor outputs: output dataset to train NN
            int num_epochs: iterations of training
        '''
        oputputs = outputs.to(torch.float32)
        inputs = inputs.to(torch.float32)
        for param in self.hidden1.parameters():
            param.requires_grad = False
        for param in self.hidden2.parameters():
            param.requires_grad = False
        for param in self.hidden3.parameters():
            param.requires_grad = False
        for param in self.hidden4.parameters():
            param.requires_grad = False
        for param in self.hidden5.parameters():
            param.requires_grad = False
        for param in self.output.parameters():
            param.requires_grad = False   
        self.train()


        for epoch in range(num_epochs):
            self.optimizer.zero_grad()

            predictions = self(inputs)
            loss = self.criterion(predictions, outputs)
            loss.backward()

            self.optimizer.step()

            current_loss = loss.item()
            self.loss_data['loss'].append(loss.detach().numpy())
            self.loss_data['epoch_count'].append(epoch)
            self.scheduler.step(current_loss)

            if(epoch+1) % (num_epochs/10) == 0:
                print(f'Comb NN: Epoch [{epoch+1}/{num_epochs}], Loss:{loss.item():.6f}')
                print("w2,b2",self.w2,self.b2)

    def test_model(self, inputs, outputs):
        '''
        args:
            tensor inputs: an input dataset to pass through NN and test
            tensor outputs: an output dataset to pass through NN
        '''
        inputs = inputs.to(torch.float32)
        outputs = outputs.to(torch.float32)
        self.eval()
        with torch.no_grad():
            predictions = self(inputs)
            loss = self.criterion(predictions, outputs).item()

            print(f'Test Loss: {loss:.4f}')


    def predict(self, inputs):
        '''
        args:
            tensor inputs
        returns:
            numpy array with predictions
        '''
        inputs = inputs.to(torch.float32)
        self.eval()
        with torch.no_grad():
            output = self(inputs)
            predictions = output.detach().numpy()

        return predictions



<h2> Normalize experimental data with AffineInput </h2>

In [ ]:
# Apply normalization to the training and test sets from experiments
norm_expt_training_set_df = expt_training_set_df.copy()
norm_expt_training_set_df[['z_target_um', 'TOD_fs3','GVD']] = input_transform( torch.tensor( expt_training_set_df[['z_target_um', 'TOD_fs3','GVD']].values ) )
norm_expt_training_set_df[['n_protons']] = output_transform( torch.tensor( expt_training_set_df[['n_protons']].values.reshape(-1,1) ) )


norm_expt_test_set_df = expt_test_set_df.copy()
norm_expt_test_set_df[['z_target_um', 'TOD_fs3','GVD']] = input_transform( torch.tensor( expt_test_set_df[['z_target_um', 'TOD_fs3','GVD']].values ) )
norm_expt_test_set_df[['n_protons']] = output_transform( torch.tensor( expt_test_set_df[['n_protons']].values.reshape(-1,1) ) )




In [ ]:
fig, ax = plt.subplots()

ax.scatter(norm_sim_training_set_df['z_target_um'], norm_sim_training_set_df['n_protons'], label='Simulation training set')
ax.scatter(norm_expt_training_set_df['z_target_um'], norm_expt_training_set_df['n_protons'], label='Expt training set')
ax.scatter(norm_expt_test_set_df['z_target_um'], norm_expt_test_set_df['n_protons'], label='Expt test set')


plt.title("n_protons predictions")
plt.xlabel('z_target (m)')
plt.ylabel('Number of protons (1/sr)')
plt.legend()

<h2> Train the linear scaling parameters on the output, namely, w2, b2, on the training set of experimental data </h2>

In [ ]:
norm_expt_inputs_training = torch.tensor( norm_expt_training_set_df[['z_target_um', 'TOD_fs3','GVD']].values, dtype=torch.float)
norm_expt_outputs_training = torch.tensor( norm_expt_training_set_df['n_protons'].values.reshape(-1,1), dtype=torch.float)

norm_expt_inputs_test = torch.tensor( norm_expt_test_set_df[['z_target_um', 'TOD_fs3','GVD']].values, dtype=torch.float)
norm_expt_outputs_test = torch.tensor( norm_expt_test_set_df['n_protons'].values.reshape(-1,1), dtype=torch.float)


finetune_net = finetune_NN(sim_net,learning_rate=0.01)
finetune_net.train_model(norm_expt_inputs_training,norm_expt_outputs_training,num_epochs=10000)

In [ ]:
### Plot the loss and print the values of the learned calibration parameters 
finetune_net.plot_loss()
for name, param in finetune_net.named_parameters():
    if param.requires_grad:
        print(name, param.data)

In [ ]:
#Predictions from calibrated NN on testing data from experiments
finetune_net_pred_expt_test = finetune_net.predict(norm_expt_inputs_test)

In [ ]:
#Predictions from calibrated NN on testing data from experiments
finetune_net_pred_expt_train = finetune_net.predict(norm_expt_inputs_training)

In [ ]:
simnet_expt_train_pred = sim_net.predict(norm_expt_inputs_training)
simnet_expt_test_pred = sim_net.predict(norm_expt_inputs_test)

In [ ]:
fig, ax = plt.subplots(figsize=(10,8))

ax.scatter(norm_sim_training_set_df['z_target_um'], norm_sim_training_set_df['n_protons'], label='Simulation training set')
ax.scatter(norm_expt_training_set_df['z_target_um'], norm_expt_training_set_df['n_protons'], label='Expt training set')
ax.scatter(norm_expt_test_set_df['z_target_um'], norm_expt_test_set_df['n_protons'], label='Expt test set')

ax.scatter(norm_expt_training_set_df['z_target_um'], finetune_net_pred_expt_train, label=' expt train predictions', s=50, facecolors='none', edgecolors='m')
ax.scatter(norm_expt_test_set_df['z_target_um'], finetune_net_pred_expt_test,label=' expt test predictions', s=50, facecolors='none', edgecolors='r')

# ax.scatter(norm_expt_training_set_df['z_target_um'], simnet_expt_train_pred, label='exp train predictions from uncalibrated base NN', s=50, facecolors='none', edgecolors='cyan')
# ax.scatter(norm_expt_test_set_df['z_target_um'], simnet_expt_test_pred,label=' expt test predictions from uncalibrated base NN', s=50, facecolors='none', edgecolors='k')


plt.title("n_protons predictions")
plt.xlabel('z_target (m)')
plt.ylabel('Number of protons (1/sr)')
plt.legend()

<h2> Saving the Lume Model </h2>

In [ ]:
from lume_model.models import TorchModel
from lume_model.variables import ScalarInputVariable, ScalarOutputVariable
model = TorchModel

In [ ]:

input_variables = [
    ScalarInputVariable(name="z_target_um", default=0, value_range=[-150,150]),
    ScalarInputVariable(name="TOD_fs3", default=0, value_range=[-8e4,8e4]),
    ScalarInputVariable(name="GVD", default=0, value_range=[13.2,13.9])
]
output_variables = [
    ScalarOutputVariable(name="n_protons", default=0, value_range=[0,8e10])
]

#Storing the weights and biases that calibrate the output
output_weight_params = []
output_bias_params = []

# Iterate over the model's parameters
for name, param in finetune_net.named_parameters():
    # Check if the parameter requires gradients and its name contains 'w'
    if param.requires_grad:
        print(name,param)
    if param.requires_grad and 'w' in name.lower():
        # Append the parameter's name to the array
        output_weight_params.append(param.data)
    if param.requires_grad and 'b' in name.lower():
        # Append the parameter's name to the array
        output_bias_params.append(param.data)


##### Here as well -- assuming that we know that param_stored[0] is w, and param_stored[1] is bias
calibration_transform = AffineInputTransform( 1, coefficient=torch.tensor(output_weight_params), offset=torch.tensor(output_bias_params))

model = TorchModel(
    model=sim_net,
    input_variables=input_variables,
    output_variables=output_variables,
    input_transformers=[input_transform],
    output_transformers=[calibration_transform,output_transform] # saving calibration before normalization
)

model.dump( file='base_simulation_model_with_transformers.yml' )


<h2> LoadingModel </h2>

In [ ]:
loaded_model = TorchModel('base_simulation_model_with_transformers.yml')

In [ ]:
plt.clf()
ax = plt.figure().add_subplot(projection='3d')


expt_training_set_df = pd.read_csv('../../experimental_data/training_set_1.csv')
expt_test_set_df = pd.read_csv('../../experimental_data/test_set_1.csv')


ax.scatter( expt_training_set_df['TOD_fs3'], expt_training_set_df['z_target_um'],
            expt_training_set_df['n_protons'], alpha=0.7, label='Expt Training Set')
ax.scatter( expt_test_set_df['TOD_fs3'], expt_test_set_df['z_target_um'],
            expt_test_set_df['n_protons'], alpha=0.7, label='Expt Training Set')

n_predict = loaded_model.evaluate(
    {label: torch.tensor( expt_training_set_df[label].values ) for label in ['z_target_um', 'TOD_fs3', 'GVD']})
ax.scatter( expt_training_set_df['TOD_fs3'], expt_training_set_df['z_target_um'],
           n_predict['n_protons'], s=50, facecolors='none', edgecolors='r', label='prediction on training set')


n_predict = loaded_model.evaluate(
    {label: torch.tensor( expt_test_set_df[label].values ) for label in ['z_target_um', 'TOD_fs3', 'GVD']})
ax.scatter( expt_test_set_df['TOD_fs3'], expt_test_set_df['z_target_um'],
           n_predict['n_protons'], s=50, facecolors='none', edgecolors='r', label='prediction on test set')

ax.view_init(elev=40., azim=40, roll=0)
plt.xlabel('TOD')
plt.ylabel('z_target')
plt.legend()

In [ ]:
plt.clf()
ax = plt.figure().add_subplot()


expt_training_set_df = pd.read_csv('../../experimental_data/training_set_1.csv')
expt_test_set_df = pd.read_csv('../../experimental_data/test_set_1.csv')

ax.scatter( expt_training_set_df['z_target_um'],
            expt_training_set_df['n_protons'], alpha=0.7, label='Expt train set')
ax.scatter( expt_test_set_df['z_target_um'],
           expt_test_set_df['n_protons'], alpha=0.7, label='Expt test set')


n_predict = loaded_model.evaluate(
    {label: torch.tensor( expt_training_set_df[label].values ) for label in ['z_target_um', 'TOD_fs3', 'GVD']})
ax.scatter( expt_training_set_df['z_target_um'],
           n_predict['n_protons'], s=50, facecolors='none', edgecolors='r', label='Prediction on expt train set')


n_predict = loaded_model.evaluate(
    {label: torch.tensor( expt_test_set_df[label].values ) for label in ['z_target_um', 'TOD_fs3', 'GVD']})
ax.scatter( expt_test_set_df['z_target_um'],
           n_predict['n_protons'], s=50, facecolors='none', edgecolors='b', label='Prediction on expt test set')


plt.xlabel('TOD')
plt.ylabel('z_target')
plt.legend()